# Generate collaboration metrics

In [2]:
import pandas as pd
from pathlib import Path, PurePath
import os
import sys
import numpy as np

In [3]:
from github2pandas_manager.config_parser import YAML_RequestDefinition
import aux

In [4]:
sys.path.append(os.path.join(sys.path[0],'..','..','FeedbackGeneration'))
from DataHandler import DataHandler

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
result_folder = "./example/notebooks/"
result_file_name = "F_CollaborationRatio"

In [15]:
project_folder_name = "../"
project_parameter_file = "config.yml"

In [16]:
# Just for local testing purposes 
# project_folder_name = "./example/"

In [19]:
request_params = YAML_RequestDefinition(project_folder_name+project_parameter_file)
print(request_params.parameters.project_folder)

./example/data/TaskExamples


In [18]:
path = PurePath(os.getcwd())
if path.name == "notebooks":
    print(os.getcwd())
    os.chdir("../../")
    print(os.getcwd())

/home/zug/Desktop/github2pandas/TutorFeedback_StudentRepos_5/example/notebooks
/home/zug/Desktop/github2pandas/TutorFeedback_StudentRepos_5


In [20]:
dh = DataHandler(request_params.parameters.project_folder)

example/data/TaskExamples


In [21]:
dh.raw_pdCommits['team'] =  dh.raw_pdCommits['repo_name'].apply(aux.getTeamName)
dh.raw_pdEdits['team'] =  dh.raw_pdEdits['repo_name'].apply(aux.getTeamName)
dh.raw_pdIssues['team'] =  dh.raw_pdIssues['repo_name'].apply(aux.getTeamName)
dh.raw_pdRepositories['team'] =  dh.raw_pdRepositories['repo_name'].apply(aux.getTeamName)
dh.raw_pdUsers['team'] =  dh.raw_pdUsers['repo_name'].apply(aux.getTeamName)

In [22]:
github_users_black_list = ["github-classroom[bot]", "actions-user", "web-flow", "github-actions[bot]", "GitHubAction"]
github_users_black_list_ids = dh.raw_pdUsers [dh.raw_pdUsers.login.isin(github_users_black_list)].anonym_uuid.unique().tolist() 

tutor_black_list = ["SebastianZug", "andre-dietrich", "JayTee42", "galinarudollf", "bpaulwitz",
                    "Sebastian Zug", "gjaeger1", "Florian Richter", "RiFlorian", "JayTee"]
tutor_black_list_ids = dh.raw_pdUsers [dh.raw_pdUsers.login.isin(tutor_black_list)].anonym_uuid.unique().tolist() 

black_list_ids = tutor_black_list_ids + github_users_black_list_ids
black_list_ids

['can-expect-late-month', 'would-hot-power-president']

In [23]:
dh.raw_pdUsers['role'] = "student"
dh.raw_pdUsers.loc[dh.raw_pdUsers.anonym_uuid.isin(github_users_black_list_ids), "role"] = 'github'
dh.raw_pdUsers.loc[dh.raw_pdUsers.anonym_uuid.isin(tutor_black_list_ids), "role"] = 'tutor'

In [24]:
df = pd.merge(dh.raw_pdEdits, dh.raw_pdCommits[['commit_sha', 'author']], how='left', left_on=['commit_sha'], right_on = ['commit_sha'])
df = pd.merge(df, dh.raw_pdUsers[['anonym_uuid', 'role']], how='left', left_on=['author'], right_on = ['anonym_uuid'])

In [29]:
df_files = df[df.role != "github"].groupby(['team', 'author']).filename.unique().to_frame()
df_files_separated = pd.DataFrame(df_files.to_records(),
                                  columns=df_files.index.names + list(df_files.columns))

teams = df_files_separated.groupby('team').author.unique()
dict_teams =teams.to_dict()

In [30]:
dict_teams

{'42': array(['kill-ask-poor-service', 'let-other-medical-state'], dtype=object),
 '_blank_': array(['become-new-city-house', 'seem-religious-car-family'], dtype=object),
 'aufgabe3': array(['use-clear-low-night'], dtype=object),
 'computerministers': array(['see-should-serious-person'], dtype=object),
 'hier_kreativen_namen_einfugen': array(['pay-walk-short-day', 'see-environmental-low-hand'], dtype=object),
 'ichhabkeinteam': array(['like-past-cold-girl', 'tell-long-mother-health'], dtype=object),
 'irgendwas': array(['hold-get-other-fact', 'stay-general-main-money'], dtype=object),
 'linux': array(['tell-environmental-clear-home'], dtype=object),
 'nobodyexpectsthespanishinquisition': array(['know-live-big-right', 'read-main-serious-party'], dtype=object),
 'plantpoweredprogrammers': array(['bring-cold-common-face', 'put-understand-possible-education'],
       dtype=object),
 'poorgrammer': array(['lead-natural-full-family'], dtype=object),
 'suppengrun': array(['be-full-certain-hom

In [35]:
df_files = df[df.role != "github"].groupby(['team', 'author']).filename.unique().to_frame()
df_files_separated = pd.DataFrame(df_files.to_records(),
                                  columns=df_files.index.names + list(df_files.columns))

count = 0
authors_list =  ['author_A', 'author_B']
df['authorID'] = "Orga"
for index, row in df[df.role != "github"].iterrows():
    members = dict_teams[row.team]
    if row.author in members:
        idx = list(members).index(row.author)
        df.at[index, 'authorID'] = authors_list[idx]

pdResults = df.groupby(['team', 'authorID']).filename.unique().unstack()
pdResults

authorID,Orga,author_A,author_B
team,,,
42,"[startYourImplementationHere, README.md, team....","[team.config, Program.cs]","[.gitignore, Aufgabe_2.csproj, Program.cs, Auf..."
_blank_,"[startYourImplementationHere, README.md, team....",[.gitignore],[team.config]
aufgabe3,"[startYourImplementationHere, README.md, team....","[README.md, Aufgabe_05, startYourImplementatio...",NaN
computerministers,"[team.config, startYourImplementationHere, REA...","[Aufgabe_1.csproj, Program.cs, startYourImplem...",NaN
hier_kreativen_namen_einfugen,"[startYourImplementationHere, README.md, team....","[team.config, .gitignore, Aufgabe_1.csproj, Pr...","[.gitignore, Aufgabe_2.csproj, Program.cs]"
ichhabkeinteam,"[Aufgabe_1.csproj, Program.cs, startYourImplem...",[team.config],"[launch.json, tasks.json, Aufgabe1.code-worksp..."
irgendwas,"[startYourImplementationHere, README.md, team....","[team.config, .gitignore, A5-2.csproj, Program...","[team.config, Aufgabe_1.cs, Aufgabe_1_v2.cs]"
linux,"[startYourImplementationHere, README.md, team....","[Helm.cs, Helmtiger.cs, Program.cs, Tiger.cs, ...",NaN
nobodyexpectsthespanishinquisition,"[startYourImplementationHere, README.md, team....",[team.config],[team.config]


In [36]:
pdResults['CollaborationRatio'] = 0
for index, row in pdResults.iterrows():
    if (not isinstance(row.author_A,float)):
        A_files = list(row.author_A)
    else:
        A_files = []
    if (not isinstance(row.author_B,float)):
        B_files = list(row.author_B)
    else:
        B_files = []
    
    if A_files and B_files:
        all_files = set(A_files) | set(B_files)
    if A_files and not B_files:
        all_files = set(A_files)
    if not A_files and B_files:
        all_files = set(B_files)

    pdResults.at[index, 'CollaborationRatio'] = len(set(A_files) & set(B_files)) / len(all_files)

In [37]:
pdResults.reset_index(inplace=True)
pdResults['team'] = "`" + pdResults['team'] + "`" 
pdResults

authorID,team,Orga,author_A,author_B,CollaborationRatio
0,`42`,"[startYourImplementationHere, README.md, team....","[team.config, Program.cs]","[.gitignore, Aufgabe_2.csproj, Program.cs, Auf...",0.100000
1,`_blank_`,"[startYourImplementationHere, README.md, team....",[.gitignore],[team.config],0.000000
2,`aufgabe3`,"[startYourImplementationHere, README.md, team....","[README.md, Aufgabe_05, startYourImplementatio...",NaN,0.000000
3,`computerministers`,"[team.config, startYourImplementationHere, REA...","[Aufgabe_1.csproj, Program.cs, startYourImplem...",NaN,0.000000
4,`hier_kreativen_namen_einfugen`,"[startYourImplementationHere, README.md, team....","[team.config, .gitignore, Aufgabe_1.csproj, Pr...","[.gitignore, Aufgabe_2.csproj, Program.cs]",0.400000
5,`ichhabkeinteam`,"[Aufgabe_1.csproj, Program.cs, startYourImplem...",[team.config],"[launch.json, tasks.json, Aufgabe1.code-worksp...",0.000000
6,`irgendwas`,"[startYourImplementationHere, README.md, team....","[team.config, .gitignore, A5-2.csproj, Program...","[team.config, Aufgabe_1.cs, Aufgabe_1_v2.cs]",0.285714
7,`linux`,"[startYourImplementationHere, README.md, team....","[Helm.cs, Helmtiger.cs, Program.cs, Tiger.cs, ...",NaN,0.000000
8,`nobodyexpectsthespanishinquisition`,"[startYourImplementationHere, README.md, team....",[team.config],[team.config],1.000000
9,`plantpoweredprogrammers`,"[startYourImplementationHere, README.md, team....","[aufgabe1-2, aufgabe1-2.cs]",[startYourImplementationHere],0.000000


In [38]:
with open(result_folder + result_file_name + '.txt', 'w') as f:
    f.write(pdResults[['team', 'author_A', 'author_B', 'CollaborationRatio']].to_markdown())